In [ ]:
import sys
sys.path.insert(0,'../src/')

from data.make_dataset import *
from visualization.visualize import *
from models.models import *

%load_ext autoreload
%autoreload 1

In [ ]:
channel_size = 3
image_size = 64
batch_size = 128
workers = 10

In [ ]:
data_loader = make_dataloader("../data/raw/planctons_original", image_size, batch_size, workers)

In [ ]:
print_samples(data_loader)

In [ ]:
latent_size = 100
feature_map_size = 64

beta = 0.5
lr = 0.0002
num_epochs = 200

In [ ]:
g = Generator(latent_size, feature_map_size, channel_size)
d = Discriminator(feature_map_size, channel_size)

gan = DCGAN(d, g, lr, beta)

In [ ]:
gan.train(data_loader, 1)